# Data Cleaning & Text Pre-Processing on Amazon Food Reviews

Dataset Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of the fine food from Amazon.

Timespan: Oct 1999 - Oct 2012
Total Number of Reviews: 568,454
Total Number of Users: 256,059
Total Number of Products: 74,258
Total Number of Profile Name: 218,418
?
Number of attributes/columns: 10

Attributes/Columns:

1. Id: Row Id
2. ProductId: Unique identifier for the product
3. UserId: Unqiue identifier for the user
4. ProfileName: Profile name of the user
5. HelpfulnessNumerator: Number of users who found the review helpful
6. HelpfulnessDenominator: Number of users who indicated whether they found the review helpful or not
7. Score: Rating between 1 and 5
8. Time: Timestamp for the review
9. Summary: Brief summary of the review
10. Text: Text of the review

**Aim: On Amazon Fine Food Review dataset, perform data cleaning and text pre-processing on the text of reviews.And create a database that includes PreProcessed text column that include cleaned and preprocessed text.**

# Loading the data

The dataset is available in two forms on kaggle:

1. .csv file

2. SQLite Database

To load data, I have used SQLite dataset as it is easier to query and visualise the data.Here I have to classifiy the sentiment into positive and negative, so I will ignore all the reviews with Scores equal to 3.If the Score is greater than 3 then it is positive otherwise it is negative.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer

In [2]:
conn=sqlite3.connect('database.sqlite')

In [3]:
conn

In [4]:
data=pd.read_sql_query("SELECT * FROM REVIEWS WHERE Score!=3",conn)

In [5]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
data.shape

(525814, 10)

# Data Cleaning

Dataset have many duplicate entries,So I have removed all the duplicate entries to get the unbiased results.

In [7]:
data['UserId'].count()

525814

In [8]:
data.groupby('UserId')['UserId'].nunique()

UserId
#oc-R103C0QSV1DF5E    1
#oc-R109MU5OBBZ59U    1
#oc-R10LFEMQEW6QGZ    1
#oc-R10UA029WVWIUI    1
#oc-R115TNMSPFT9I7    1
#oc-R119LM8D59ZW8Y    1
#oc-R11D9D7SHXIJB9    1
#oc-R11DNU2NBKQ23Z    1
#oc-R11O5J5ZVQE25C    1
#oc-R120LO6OLNDPCG    1
#oc-R12KPBODL2B5ZD    1
#oc-R12MGTQS5KZZRV    1
#oc-R12N3533IO3B79    1
#oc-R13EBF129DBX88    1
#oc-R13NNUL4EKL4FL    1
#oc-R13X3YIJ6GLT0C    1
#oc-R1493EDK6REQ02    1
#oc-R149FDXLRARCWJ    1
#oc-R14VLCY75KIBSB    1
#oc-R14ZSRYW2YB41B    1
#oc-R14ZUK54VMOGJS    1
#oc-R1522DF2LUL4G1    1
#oc-R152UR09M996EM    1
#oc-R15343ZW0UTLMR    1
#oc-R155JB2SA58E17    1
#oc-R15OCG1KF51S4F    1
#oc-R162D7S0A880MV    1
#oc-R163CP16SRRI50    1
#oc-R1669TSD0WO7EP    1
#oc-R1730HL4795MHX    1
                     ..
AZZFEH7BWEWRD         1
AZZFJQFHITBZ5         1
AZZFLKZ198VZV         1
AZZFRY8O72WH8         1
AZZH3GGYQSBUC         1
AZZHZZMH3U1VB         1
AZZJDUEFXYXBM         1
AZZLMJ88R9HPH         1
AZZMDW27MUJR6         1
AZZMO52V8WZ68         1
AZZNK89PX

In [9]:
dataDuplicationCheck=pd.read_sql_query("SELECT * FROM REVIEWS WHERE Score!=3 AND UserId='AR5J8UI46CURR' ORDER BY 'ProductId'",conn)

In [10]:
dataDuplicationCheck

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


Soretd the dataset on the basis of ProductId

In [11]:
dataSort=data.sort_values('ProductId',inplace=False)

In [12]:
dataSort.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,4,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,4,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,5,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,5,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


In [13]:
afterRemovingDuplicates=dataSort.drop_duplicates(subset=["UserId","ProfileName","Time","Text"],keep='first',inplace=False)

In [14]:
afterRemovingDuplicates.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,4,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,4,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,5,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,5,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


The row having HelpfulnessNumerator grater than HelpfulnessDenominator is not possible practically so I have removed that all rows.

In [15]:
helpfulness=afterRemovingDuplicates[afterRemovingDuplicates.HelpfulnessNumerator>afterRemovingDuplicates.HelpfulnessDenominator]

In [16]:
helpfulness.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
59301,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
41159,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [17]:
cleanData=afterRemovingDuplicates[afterRemovingDuplicates.HelpfulnessNumerator<=afterRemovingDuplicates.HelpfulnessDenominator]

In [18]:
cleanData.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,4,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,4,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,5,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,5,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


# Changing Score to Positive and Negative

In [19]:
def change_score(score):
    if(score>3):
        return "Positive"
    else:
        return "Negative"

In [20]:
scoreList=list(cleanData['Score'])

In [21]:
scoreList[:10]

[5, 4, 4, 5, 5, 4, 5, 5, 5, 5]

In [22]:
changedScore=list(map(change_score,scoreList))

In [23]:
changedScore[:10]

['Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive']

In [24]:
cleanData['Score']=changedScore

C:\Users\Vaibhav\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
cleanData['Score'].head()

138706    Positive
138688    Positive
138689    Positive
138690    Positive
138691    Positive
Name: Score, dtype: object

In [26]:
finalCleanData=cleanData.sort_index()

In [27]:
finalCleanData.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,Positive,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...


In [28]:
finalCleanData['Score'].value_counts()

Positive    307061
Negative     57110
Name: Score, dtype: int64

# Text Preprocessing

In the Preprocessing phase :-

1. Removed the html tags(Anything which is in "<>" braces will be removed).
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming).
7. After which we collect the words used to describe positive and negative reviews

**Here I have not removed stop words from review text as I want more accurate prediction and separation of positive and negative reviews**

In [29]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [43]:
def cleanhtml(sentence):
    cleaner=re.compile("<.*?>")
    cleanText=re.sub(cleaner," ",sentence)
    return cleanText

In [44]:
def cleanpunc(sentence):
    cleanText=re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleanText=re.sub(r'[.|,|)|(|\|/]',r'',cleanText)
    return cleanText

In [45]:
snow=nltk.stem.SnowballStemmer('english')

In [46]:
final_text=[]
s=""
for sentence in finalCleanData["Text"].values:
    filteredSentence=[]
    eachReviewText=""
    htmlCleanedSentence=cleanhtml(sentence)
    for word in htmlCleanedSentence.split():
        for puncCleanedSentence in cleanpunc(word).split():
            if((puncCleanedSentence.isalpha()) & (len(puncCleanedSentence)>2)):
                sentenceLower=puncCleanedSentence.lower()
                s=snow.stem(sentenceLower)
                filteredSentence.append(s)
                
    eachReviewText=' '.join(filteredSentence)
    final_text.append(eachReviewText)
    

In [47]:
finalCleanData["ProcessedText"]=final_text

In [48]:
finalCleanData["ProcessedText"].head()

0    have bought sever the vital can dog food produ...
1    product arriv label jumbo salt peanutsth peanu...
2    this confect that has been around few centuri ...
4    great taffi great price there was wide assort ...
5    got wild hair for taffi and order this five po...
Name: ProcessedText, dtype: object

In [49]:
finalCleanData.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,ProcessedText
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,have bought sever the vital can dog food produ...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arriv label jumbo salt peanutsth peanu...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,this confect that has been around few centuri ...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffi great price there was wide assort ...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,Positive,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,got wild hair for taffi and order this five po...


In [50]:
finalCleanData.shape

(364171, 11)

In [54]:
connection=sqlite3.connect('CleanedAmazonFoodReviewDataset.sqlite')
finalCleanData.to_sql('Reviews', connection, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

In [56]:
checkingData=pd.read_sql_query("SELECT * FROM Reviews",connection)
checkingData.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,ProcessedText
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,have bought sever the vital can dog food produ...
1,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arriv label jumbo salt peanutsth peanu...
2,2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,this confect that has been around few centuri ...
3,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffi great price there was wide assort ...
4,5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,Positive,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,got wild hair for taffi and order this five po...


In [58]:
checkingData.shape

(364171, 12)